In [1]:
import pandas as pd
import numpy  as np

   
import pickle
   

from scipy.sparse import issparse

 
from pathlib import Path


pd.set_option('display.max_columns', 5)

In [2]:
#### user specified
data_folder = "D:/analyze_Pearson_residuals/"

data_subfolder = "10k_brain"

data_path = Path ( data_folder + data_subfolder )

In [3]:
# output data
results_pkl = "Lg_complementary_samples.pkl"

# input data
counts_pkl = "counts_sparse.pkl" # sparse pandas data frame: cells in columns; genes in rows
gene_array_pkl  = "gene_array_complementary_samples.pkl"
df_cells_pkl  = "df_cells_complementary_samples.pkl" 


# path: output data
results_dsn = data_path / results_pkl

# paths: input data
counts_dsn = data_path / counts_pkl  
df_cells_dsn = data_path / df_cells_pkl
gene_array_dsn = data_path / gene_array_pkl

In [4]:
# https://github.com/scverse/scanpy/tree/master/scanpy/experimental/pp
# from   _normalization.py  rows 26-64

def _pearson_residuals(X, theta, clip, copy=False):    # removed check_values from parm list

    X = X.copy() if copy else X

    # check theta
    if theta <= 0:
        # TODO: would "underdispersion" with negative theta make sense?
        # then only theta=0 were undefined..
        raise ValueError('Pearson residuals require theta > 0')
    # prepare clipping
    if clip is None:
        n = X.shape[0]
        clip = np.sqrt(n)
    if clip < 0:
        raise ValueError("Pearson residuals require `clip>=0` or `clip=None`.")

# vfk 2022 08 16		
    # if check_values and not check_nonnegative_integers(X):
        # warn(
            # "`normalize_pearson_residuals()` expects raw count data, but non-integers were found.",
            # UserWarning,
        # )

    if issparse(X):
        sums_genes = np.sum(X, axis=0)
        sums_cells = np.sum(X, axis=1)
        sum_total = np.sum(sums_genes).squeeze()
    else:
        sums_genes = np.sum(X, axis=0, keepdims=True)
        sums_cells = np.sum(X, axis=1, keepdims=True)
        sum_total = np.sum(sums_genes)

    mu = np.array(sums_cells @ sums_genes / sum_total)
    diff = np.array(X - mu)
    residuals = diff / np.sqrt(mu + mu**2 / theta)

    # clip
    residuals = np.clip(residuals, a_min=-clip, a_max=clip)

    return residuals	
	
	
pctl_list = [.01,.05, .10, .25, .5, .75, .90, .95, .96, .97, .98, .99 ]

In [5]:
df_selected_cells = pd.read_pickle( df_cells_dsn )
print ( ' df_selected_cells:  ' )
print ( '\n', df_selected_cells )
print ( '\n\n df_selected_cells.sum:  ' )
print ( df_selected_cells.sum() )

f = open( gene_array_dsn, 'rb' )    
gene_select_array  = pickle.load(f)     
f.close()       
print ( '\n\n gene_select_array:  ' )
print ( gene_select_array )

 df_selected_cells:  

                         0      1
AAACCCAAGCAACTCT-1  False   True
AAACCCACACGCGGTT-1   True  False
AAACCCACAGCATACT-1   True  False
AAACCCACATACCATG-1   True  False
AAACCCAGTCGCACAC-1  False   True
...                   ...    ...
TTTGTTGGTAGCTAAA-1  False   True
TTTGTTGGTATCCCAA-1  False   True
TTTGTTGGTCCGAAAG-1  False   True
TTTGTTGGTTCAACGT-1   True  False
TTTGTTGTCCGTTTCG-1  False   True

[11843 rows x 2 columns]


 df_selected_cells.sum:  
0    5980
1    5863
dtype: int64


 gene_select_array:  
['ENSMUSG00000052305' 'ENSMUSG00000073940' 'ENSMUSG00000069919' ...
 'ENSMUSG00000020930' 'ENSMUSG00000027956' 'ENSMUSG00000027075']


In [6]:
df_counts_sparse = pd.read_pickle ( counts_dsn )
print ( ' df_counts_sparse:' )
print ( df_counts_sparse )
print ( '\n type( df_counts_sparse ) ', type ( df_counts_sparse ))
print ( '\n df_counts_sparse.sparse.density: ', df_counts_sparse.sparse.density)

 df_counts_sparse:
                    AAACCCAAGCAACTCT-1  AAACCCACACGCGGTT-1  ...  \
ENSEMBL_ID                                                  ...   
ENSMUSG00000051951                   0                   0  ...   
ENSMUSG00000025900                   0                   0  ...   
ENSMUSG00000025902                   0                   0  ...   
ENSMUSG00000033845                   0                   2  ...   
ENSMUSG00000025903                   0                   0  ...   
...                                ...                 ...  ...   
ENSMUSG00000094350                   0                   0  ...   
ENSMUSG00000079808                   0                   0  ...   
ENSMUSG00000095041                   0                   0  ...   
ENSMUSG00000063897                   0                   0  ...   
ENSMUSG00000095742                   0                   0  ...   

                    TTTGTTGGTTCAACGT-1  TTTGTTGTCCGTTTCG-1  
ENSEMBL_ID                                      

In [7]:
df_counts_sparse_all_samples_SD_positive = df_counts_sparse[ df_selected_cells.index ].loc [ gene_select_array ]
print ( ' df_counts_sparse_all_samples_SD_positive:' )
print ( df_counts_sparse_all_samples_SD_positive )
print ( '\n type( df_counts_sparse_all_samples_SD_positive ) ', type ( df_counts_sparse_all_samples_SD_positive ))
print ( '\n df_counts_sparse_all_samples_SD_positive.sparse.density: ', df_counts_sparse_all_samples_SD_positive.sparse.density)

del df_counts_sparse

 df_counts_sparse_all_samples_SD_positive:
                    AAACCCAAGCAACTCT-1  AAACCCACACGCGGTT-1  ...  \
ENSEMBL_ID                                                  ...   
ENSMUSG00000052305                   0                   0  ...   
ENSMUSG00000073940                   0                   0  ...   
ENSMUSG00000069919                   0                   0  ...   
ENSMUSG00000069917                   0                   0  ...   
ENSMUSG00000019874                  20                   2  ...   
...                                ...                 ...  ...   
ENSMUSG00000070044                   0                   0  ...   
ENSMUSG00000001260                   0                   0  ...   
ENSMUSG00000020930                   0                   0  ...   
ENSMUSG00000027956                   0                   0  ...   
ENSMUSG00000027075                   0                   0  ...   

                    TTTGTTGGTTCAACGT-1  TTTGTTGTCCGTTTCG-1  
ENSEMBL_ID              

In [8]:
df_clipped_half_cell_samples_list = []

for sample in  range(2):
  print ( '\n sample: ', sample )
  df_cell_sample = df_selected_cells[[sample]]
  df_cell_select = df_cell_sample [ df_cell_sample[sample] ]
  barcode_list = df_cell_select.index.values.tolist()
  
  df_counts_sparse_sample = df_counts_sparse_all_samples_SD_positive[ barcode_list ]     
  scipy_csr_mat = df_counts_sparse_sample.transpose().sparse.to_coo().tocsr(). astype ( 'float64'  )

  residuals = _pearson_residuals ( scipy_csr_mat, 100, None )
  residual_variance_clipped = np.var( residuals, axis=0)

  df_clipped_half_cell_sample = pd.DataFrame ( index = df_counts_sparse_all_samples_SD_positive.index, data = { sample: residual_variance_clipped  } ) 
  df_clipped_half_cell_sample.sort_values ( [sample], ascending=False, inplace=True )  

  df_clipped_half_cell_samples_list.append ( df_clipped_half_cell_sample )
  
  
df_clipped_half_cell_samples = pd.concat ( df_clipped_half_cell_samples_list, axis=1 )
print ( '\n df_clipped_half_cell_samples' )
print ( df_clipped_half_cell_samples  )
print ( '\n\n df_clipped_half_cell_samples.describe' )
print ( df_clipped_half_cell_samples.describe ( percentiles=pctl_list ) )


 sample:  0

 sample:  1

 df_clipped_half_cell_samples
                             0           1
ENSEMBL_ID                                
ENSMUSG00000052305  180.364541  190.463192
ENSMUSG00000069919  141.031124  157.393667
ENSMUSG00000069917  133.260150  149.320511
ENSMUSG00000073940  104.043067  116.592786
ENSMUSG00000019874  103.180521  101.826492
...                        ...         ...
ENSMUSG00000099655    0.746357    0.922433
ENSMUSG00000096655    0.743417    1.200898
ENSMUSG00000029994    0.737800    0.789004
ENSMUSG00000020930    0.712552    0.799276
ENSMUSG00000110611    0.709176    0.889571

[12570 rows x 2 columns]


 df_clipped_half_cell_samples.describe
                  0             1
count  12570.000000  12570.000000
mean       1.475883      1.479818
std        3.021801      3.208476
min        0.709176      0.704070
1%         0.855963      0.856066
5%         0.912146      0.916471
10%        0.946068      0.950925
25%        1.025746      1.028185
50%        

In [9]:
df_clipped_half_cell_samples.to_pickle ( results_dsn )